# Development Environment

In [ ]:
from utils.jupyter_utils import log_cell
from utils.jupyter_utils import clear_logs

In [ ]:
clear_logs()

## Execution

In [ ]:
!python split_data.py run res/train.json --ratio=0.2 --suffix="02"

In [ ]:
!python split_data.py run res/train.json --ratio=0.002 --suffix="0002"

In [ ]:
!python split_data.py run res/train.json --ratio=0.001 --suffix="0001"

In [ ]:
%%log_cell

# 자가 테스트용
!python grape_recommender.py run \
    --song_meta_fname=res/song_meta.json \
    --genre_fname=res/genre_gn_all.json \
    --train_fname=arena_data/orig/train.json \
    --question_fname=arena_data/questions/val.json

In [ ]:
%%log_cell

# 자가 테스트용 채점
!python evaluate.py evaluate \
    --gt_fname=arena_data/answers/val.json \
    --rec_fname=arena_data/results/results.json

In [ ]:
%%log_cell

# 리더보드 제출용
!python grape_recommender.py run \
    --song_meta_fname=res/song_meta.json \
    --genre_fname=res/genre_gn_all.json \
    --train_fname=res/train.json \
    --question_fname=res/val.json

In [ ]:
%%log_cell

# 최종 제출용
!python grape_recommender.py run \
    --song_meta_fname=res/song_meta.json \
    --genre_fname=res/genre_gn_all.json \
    --train_fname=res/train.json \
    --question_fname=res/test.json

## Development

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def prepare_data(suffix=''):
    if not suffix:
        return
    import shutil
    shutil.copyfile(f"./arena_data/orig/train{suffix}.json", "./arena_data/orig/train.json")
    shutil.copyfile(f"./arena_data/orig/val{suffix}.json", "./arena_data/orig/val.json")
    shutil.copyfile(f"./arena_data/questions/val{suffix}.json", "./arena_data/questions/val.json")
    shutil.copyfile(f"./arena_data/answers/val{suffix}.json", "./arena_data/answers/val.json")

In [ ]:
# prepare_data("02")
prepare_data("0002")
# prepare_data("0001")

In [ ]:
from utils import read_json
from utils import write_json

In [ ]:
train = read_json('./arena_data/orig/train.json')
genre_gn_all = read_json('./res/genre_gn_all.json')
song_meta = read_json('./res/song_meta.json')

In [ ]:
# 제출용
train = read_json('./res/train.json')
genre_gn_all = read_json('./res/genre_gn_all.json')
song_meta = read_json('./res/song_meta.json')

In [ ]:
%%log_cell
%%time
import lib.graph
%aimport -lib.graph.core
%aimport -lib.graph.nodes

graph = lib.graph.GraphBuilder().build(song_meta, genre_gn_all, train)

In [ ]:
%%log_cell
%%time
import lib.grape
from constants import NUM_OF_RECOMMENDED_SONGS
from constants import NUM_OF_RECOMMENDED_TAGS

grape = lib.grape.Grape(
    graph,
    NUM_OF_RECOMMENDED_SONGS,
    NUM_OF_RECOMMENDED_TAGS,
)

In [ ]:
%%log_cell
%%time
grape.fit(train)

In [ ]:
questions = read_json('./arena_data/questions/val.json')

In [ ]:
# 제출용
questions = read_json('./res/val.json')

In [ ]:
%%log_cell
from utils import validate_answers

n_questions = 2000
# n_questions = 200
# n_questions = 100
answers = grape.predict_all(questions[:n_questions])
validate_answers(answers, questions[:n_questions])
write_json(answers, './arena_data/results/results.json')

!python evaluate.py evaluate \
    --gt_fname=arena_data/answers/val.json \
    --rec_fname=arena_data/results/results.json \
    --question_fname=arena_data/questions/val.json

In [ ]:
%%log_cell
def predict_multi(questions):
    import multiprocessing as mp
    import numpy as np
    import itertools

    jobs = mp.cpu_count()
    jobs = 2
    chunks = list(np.array_split(questions, jobs))

    with mp.Pool() as pool:
        answers = pool.map(predict_job, chunks)
        return list(itertools.chain(*answers))


def predict_job(q):
    return grape.predict_all(q)

In [ ]:
%%log_cell
from utils import validate_answers

n_questions = 2000000000
# n_questions = 200
# n_questions = 100
# answers = grape.predict_all(questions[:n_questions])
answers = predict_multi(questions[:n_questions])
validate_answers(answers, questions[:n_questions])
write_json(answers, './arena_data/results/results.json')

# !python evaluate.py evaluate \
#     --gt_fname=arena_data/answers/val.json \
#     --rec_fname=arena_data/results/results.json \
#     --question_fname=arena_data/questions/val.json

## Test

In [ ]:
%%time
import lib.graph
%aimport -lib.graph.core
%aimport  lib.graph.core.graph
%aimport -lib.graph.nodes

graph = lib.graph.GraphBuilder().build(song_meta[:100], genre_gn_all, [])

In [ ]:
%%time
import lib.grape
from constants import NUM_OF_RECOMMENDED_SONGS
from constants import NUM_OF_RECOMMENDED_TAGS

grape = lib.grape.Grape(
    graph,
    NUM_OF_RECOMMENDED_SONGS,
    NUM_OF_RECOMMENDED_TAGS,
)

In [ ]:
grape.fit(train)

In [ ]:
questions = read_json('./arena_data/questions/val.json')

In [ ]:
from utils import validate_answers

n_questions = 20
answers = grape.predict_all(questions[:n_questions])
validate_answers(answers, questions[:n_questions])
write_json(answers, './arena_data/results/results.json')

!python evaluate.py evaluate \
    --gt_fname=arena_data/answers/val.json \
    --rec_fname=arena_data/results/results.json

## Playground